In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupShuffleSplit

from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts
import run_expts

#TS Fresh Parameter Settings
from tsfresh.feature_extraction import ComprehensiveFCParameters
from tsfresh.feature_extraction import MinimalFCParameters
from tsfresh.feature_extraction import EfficientFCParameters
from tsfresh.feature_extraction import extract_features

import matplotlib.patches as mpatches

# Set display options to show all rows and columns
pd.set_option('display.max_rows', 3000)  # Show rows
pd.set_option('display.max_columns', 160)  # Show columns

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


I think I should test the random forest random seed. As silly as that sounds? Incase by chance that is impacting results

In [2]:
# Edit the grids based on best hyperparameters per Hyperparameter Tuning Analysis 5 worksheet 
RF_param_grid = { 
    'n_estimators': [1,2,3, 10],
    'max_features': [None, 'sqrt'],
    'max_depth' : [2,3,5],
    'criterion' :['gini',  'entropy'],
    'min_samples_split' : [3,4,5]}

DT_params =  {
    'min_samples_leaf': [1, 2, 3 , 5 ,10],
    'max_depth': [1, 2, 3, 5, None],
    'criterion': ["gini", "entropy"]
}

Ada_grid =  { 
    'n_estimators': [2, 3, 5, 10, 20, 40, 50, 100],
    'learning_rate': [0.01,0.05,  0.1, 0.2, 0.4, 10.0],
    }

SVC_grid = {'C': [0.1, 1, 10, 100, 1000], 
                'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
                'kernel': ['linear', 'rbf']} 

RF_param_grid = { 
    'n_estimators': [1,10],
    'max_features': ['sqrt'],
    'max_depth' : [5],
    'criterion' :['gini'],
    'min_samples_split' : [4]}

In [3]:
TS_Fresh_setting, connectivity_setting = 'Minimal' , 'beta'
RF_dict = {'RF' : GridSearchCV( RandomForestClassifier(), RF_param_grid , refit = True, verbose = 1, cv = GroupKFold(n_splits = 5)  ) }
data_type, expt_num , num_splits , clf_dict = 'N1', 1, 5,RF_dict 

#### Load the data

In [4]:
# Load the features 
t1 = time.time()

load_path = '/user/home/ko20929/work/RBD_using_custom_package/Data/freq_6_second_files/'
load_path_bpw = '/user/home/ko20929/work/RBD_using_custom_package/Data/freq_6_second_files/'
load_path_max_freqs = '/user/home/ko20929/work/RBD_using_custom_package/Execute New Experiments/Baseline_Extensions/Gen_New_Features/generated_feats/'

loaded_data = {}

X_y_groups = {}
    
X_bpw = pd.read_hdf(load_path_bpw + data_type +  'six_second_freq_df.h5', key='df', mode='r')
y = pd.read_hdf(load_path + data_type +  '_y.h5', key='df', mode='r') 
groups = pd.read_hdf(load_path + data_type +  '_groups.h5', key='df', mode='r')
X_bpw, y , groups = X_bpw.reset_index(drop = True) , y.reset_index(drop = True) , groups.reset_index(drop = True)

X_max_freqs = pd.read_hdf(load_path_max_freqs + data_type +  'six_second_max_freq_stats_df.h5', key='df', mode='r')
X_max_freqs = X_max_freqs.reset_index(drop = True)

X = pd.concat([X_bpw , X_max_freqs], axis = 1)

#Transform the X into TS_Fresh Features___
# 1. Convert to TS_Fresh format Dataframe 
ts_fresh_df = format_eeg_data.convert_sktime_df_to_ts_fresh_format(X, ts_cols = list(X.columns))

# 2. Extract TS_Fresh Features from the dataframe
if TS_Fresh_setting == 'Minimal':
    settings = MinimalFCParameters()

extracted_ts_fresh_df = extract_features(ts_fresh_df, column_id = 'id' , column_sort = 'time', default_fc_parameters=settings)

# 3. Asign extract_ts_fresh_df to the variable X
X = extracted_ts_fresh_df.copy()
print(len(X.columns))
#Drop columns where all values are NA 
X = X.dropna(axis = 1)
print(len(X.columns))

#Drop columns where all values are the same
# Find columns where all values are the same
same_value_columns = X.columns[X.nunique() == 1]
# Drop columns with the same values
X = X.drop(columns=same_value_columns)
print(len(X.columns))

#___________load the appropriate connectivity features don't concatenate to non connectivity features until after dictionary generation

connectivity_folder = '/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/Connectivity/'
X_connectivity = pd.read_hdf(connectivity_folder + data_type+ '_pli__df.h5')

X_connectivity = X_connectivity[[col for col in X_connectivity.columns if connectivity_setting in col]]

#Generate a region to features dictionary - this will enable us to run expts regionally as before
regional_features_dict = {}
region_channel_dict = constants.region_to_channel_dict
regions = list(region_channel_dict.keys())
for region in regions:
    region_features = [col for col in X.columns if region + '_' in col]
    if len(region_features) > 0 : 
        regional_features_dict[region] = region_features + list(X_connectivity.columns)

#Now concatenate the regional frequency features dataframe with the connectivity dataframe
X = X.reset_index(drop=True)
X_connectivity = X_connectivity.reset_index(drop=True)
X = pd.concat([X , X_connectivity], axis = 1)

t2 = time.time()
t2-t1

23-Oct-23 18:35:16 | WARNING | Dependency not available for matrix_profile, this feature will be disabled!
Feature Extraction: 100%|██████████| 120/120 [00:03<00:00, 32.93it/s]


1950
1950
1948


6.843061685562134

In [5]:
#Choose one region , 0 = Prefrontal
num = 0
key = list(regional_features_dict.keys())[0]
subset_dict =  {key : regional_features_dict[key] }

In [6]:
#Run the experiment with hyperparameter tuning
X_expt , y_expt , groups_expt, expt_info = run_expts.generate_expt_x_y_groups(X,y,groups, expt_num)

In [7]:
t1 = time.time()
results_df = run_expts.run_mv_tsc(X_expt,y_expt,groups_expt,clf_dict, return_df = True , subset_names_and_cols = subset_dict, random_states = [1,2], groups_for_fit = True, best_params = True)
t2 = time.time()
t2-t1

Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Fitting 5 folds for each of 2 candidates, totalling 10 fits


2.32401704788208

In [8]:
results_df

,random_state,y_true,Prefrontal_RF_y_preds,y_train,Prefrontal_RF_best_clf_params,Prefrontal_RF_cv_results,Prefrontal_RF_y_train_preds
0,1,"[[0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 0, 1, 1, 1, ...","[[0, 0, 1, 0, 0, 0, 1, 1], [0, 0, 0, 1, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[{'criterion': 'gini', 'max_depth': 5, 'max_fe...",mean_fit_time std_fit_time mean_score_tim...,"[[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,..."
1,2,"[[0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 1, 1, ...","[[0, 0, 1, 0, 0, 1, 1, 1], [0, 0, 0, 0, 1, 0, ...","[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...","[{'criterion': 'gini', 'max_depth': 5, 'max_fe...",mean_fit_time std_fit_time mean_score_tim...,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."


In [ ]:
res_df = run_expts.generate_subset_acc_std(results_df.iloc[:,:3], return_df = True)
acc_df = res_df[[col for col in res_df.columns if 'acc' in col]]
mean_acc_df = pd.DataFrame( acc_df.mean(axis=0) ).T
mean_acc_df

In [ ]:
# 9.7 seconds vs. 12.94 seconds????